# Create a DataFrame for all possible combination

In [13]:
import pandas as pd
df = pd.read_csv("Data/cosmetic_all.csv")
df.head()

,Unnamed: 0,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,Cleanser,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0
1,1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0
2,2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0
3,3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0
4,4,Cleanser,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 12 columns):
Unnamed: 0     1699 non-null int64
Label          1699 non-null object
brand          1699 non-null object
name           1699 non-null object
price          1699 non-null object
rank           1699 non-null float64
ingredients    1699 non-null object
Combination    1699 non-null float64
Dry            1699 non-null float64
Normal         1699 non-null float64
Oily           1699 non-null float64
Sensitive      1699 non-null float64
dtypes: float64(6), int64(1), object(5)
memory usage: 159.4+ KB


In [4]:
df.isna().any()

Unnamed: 0     False
Label          False
brand          False
name           False
price          False
rank           False
ingredients    False
Combination    False
Dry            False
Normal         False
Oily           False
Sensitive      False
dtype: bool

In [5]:
df.rename(columns={"Unnamed: 0": "index"}, inplace=True)

In [6]:
df.head()

,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,Cleanser,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0
1,1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0
2,2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0
3,3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0
4,4,Cleanser,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0


In [7]:
option_1 = df.Label.unique().tolist()

In [8]:
option_1

['Cleanser',
 'Treatment',
 'Serum',
 'Oils',
 'Toners',
 'Moisturizer',
 'Night Treatments',
 'Moisturiser',
 'Face Mask',
 'Eye cream',
 'Sun protect']

In [9]:
df.loc[df["Label"] == "Moisturiser", ["Label"]]= "Moisturizer"

In [10]:
type(option_1)

list

In [12]:
option_2 = df.columns[7:].tolist()

In [13]:
option_2

['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

In [121]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.manifold import TSNE

product_category = df["Label"].unique().tolist()
skin_type = ["Combination", "Dry", "Normal", "Oily", "Sensitive"]

# creating a function to put together a DataFrame of all combinations 
def combination(op_1, op_2):
    df_combi = df[df["Label"] == op_1][df[op_2] == 1]
    df_combi = df_combi.reset_index()
   
    
    # tokenize the ingredients
    ingredient_idx = {}
    corpus = []
    idx = 0
    
    for i in range(len(df_combi)):
        ingreds = df_combi["ingredients"][i]
        ingreds_lower = ingreds.lower()
        tokens = ingreds_lower.split(",")
        corpus.append(tokens)
        for ingredient in tokens:
            if ingredient not in ingredient_idx:
                ingredient_idx[ingredient] = idx
                idx +=1

                
    M = len(df_combi)
    N = len(ingredient_idx)
        
    matrix = np.zeros((M,N))

    def count_tokens(tokens):
        x = np.zeros(N)
        for t in tokens:
            idx = ingredient_idx[t]
            x[idx] = 1
        return x
    
 
    i = 0
    for tokens in corpus:
        matrix[i, :] = count_tokens(tokens)
        i += 1
        
 
    tsne = TSNE(n_components = 2, learning_rate = 200)
    tsne_features = tsne.fit_transform(matrix)

    
    df_combi["X"] = tsne_features[:, 0]
    df_combi["Y"] = tsne_features[:, 1]
    
    return df_combi

In [117]:
# df_combi = pd.DataFrame()

# for op_1 in product_category:
#     for op_2 in skin_type:
#         temp = combination(op_1, op_2)
#         temp["Label"] = op_1 + "_" + op_2
#         df_combi = pd.concat([df_combi, temp])

In [125]:
df_combi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5349 entries, 0 to 77
Data columns (total 15 columns):
level_0        5349 non-null int64
index          5349 non-null int64
Label          5349 non-null object
brand          5349 non-null object
name           5349 non-null object
price          5349 non-null object
rank           5349 non-null float64
ingredients    5349 non-null object
Combination    5349 non-null float64
Dry            5349 non-null float64
Normal         5349 non-null float64
Oily           5349 non-null float64
Sensitive      5349 non-null float64
X              5349 non-null float32
Y              5349 non-null float32
dtypes: float32(2), float64(6), int64(2), object(5)
memory usage: 786.8+ KB


In [70]:
# df_combi.to_csv('Data/cosmetic_all_comb.csv', encoding = 'utf-8-sig', index = False)

# Most frequent ingredients

In [14]:
# tokenize the ingredients
ingredient_idx = {}
corpus = []
idx = 0
    
for i in range(len(df)):
    ingredients = df["ingredients"][i]
    ingredients_lower = ingredients.lower() # lower the ingredients
    tokens = ingredients_lower.split(",") # split the text
    corpus.append(tokens) # text added to corpus
    for ingredient in tokens:
        if ingredient not in ingredient_idx: # if no duplicates 
            ingredient_idx[ingredient] = idx # put it in the dictionary with index
            idx += 1 # value of index will increase one by one

In [17]:
import operator

freq_items = sorted(ingredient_idx.items(), key=operator.itemgetter(1), reverse=True)

In [18]:
top_freq_items = freq_items[:20]

In [129]:
top_freq_items

[('visit the dermaflash boutique', 7821),
 (' methylcellulose', 7820),
 (' ozonized oryza sativa (rice) callus culture extract', 7819),
 (' sodium carboxyethylsilanetriol', 7818),
 (' sodium lignosulfonate', 7817),
 (' and fight wrinkles.\r\r\n-light-diffusing spheres: provide a professionally retouched look.',
  7816),
 (' support elasticity', 7815),
 ('-pepha® (derived from watermelon extract): protects skin cells against daily stress.\r\r\n- vitasource™ and dglyage®: firm the appearance of skin',
  7814),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 7813),
 (' quaternium-18 bentonite', 7812),
 (' argania spinosa (argan) oil*. *organic.', 7811),
 (' passiflora incarnata (passionfruit) oil', 7810),
 ('prunus amygdalus dulcis (sweet almond) oil*', 7809),
 ('visit the coola boutique', 7808),
 (' carmine.', 7807),
 (' aluminum powder', 7806),
 (' chromium hydroxide green', 7805),
 (' tetrasodium edta. maycontain: mica', 7804),
 (' magnesium myristate', 7803),
 (' zinc oxi

In [19]:
top_freq_items = pd.DataFrame(top_freq_items)

In [20]:
top_freq_items

,0,1
0,visit the dermaflash boutique,7821
1,methylcellulose,7820
2,ozonized oryza sativa (rice) callus culture e...,7819
3,sodium carboxyethylsilanetriol,7818
4,sodium lignosulfonate,7817
5,and fight wrinkles.\r\r\n-light-diffusing sph...,7816
6,support elasticity,7815
7,-pepha® (derived from watermelon extract): pro...,7814
8,chlorhexidine digluconate. [+/-: iron oxides ...,7813
9,quaternium-18 bentonite,7812


# T-SNE

In [40]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# getting a dataframe for dry skin and moisturiser
# set arrays for product category and skin types
product_category = df["Label"].unique()
skin_type = ["Combination", "Dry", "Normal", "Oily", "Sensitive"]

# get the dataframe
dry_moist = df[df["Label"] == "Moisturizer"][df["Dry"] == 1]
dry_moist = dry_moist.reset_index()

In [41]:
# Tokenizing ingredients
ingredient_idx = {}
corpus = []
idx = 0

for i in range(len(dry_moist)):
    ingredients = dry_moist["ingredients"][i]
    ingredients_lower = ingredients.lower() # lower the ingredients
    tokens = ingredients_lower.split(",") # split the text
    corpus.append(tokens) # text added to corpus
    for ingredient in tokens:
        if ingredient not in ingredient_idx: # if no duplicates 
            ingredient_idx[ingredient] = idx # put it in the dictionary with index
            idx += 1 # value of index will increase one by one

In [42]:
# making document-term matrix
# each cosmetic product will correspond to one document
# first, make an empty matrix 
M = len(dry_moist)
N = len(ingredient_idx)

# fill it with zeros
matrix = np.zeros((M,N))

In [45]:
def count_tokens(tokens):
    x = np.zeros(N)
    for ingredient in tokens:
        idx = ingredient_idx[ingredient]
        x[idx] = 1
    return x

In [46]:
i = 0
for tokens in corpus:
    matrix[i,:] = count_tokens(tokens)
    i += 1

In [47]:
matrix.shape

(208, 2385)

In [48]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, learning_rate=200)
tsne_features = model.fit_transform(matrix)

In [49]:
dry_moist["X"] = tsne_features[:, 0]
dry_moist["Y"] = tsne_features[:, 1]

In [50]:
dry_moist.head()

,level_0,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
0,117,117,Moisturizer,L'Oreal Paris,L'Oreal Paris Age Perfect Anti Aging Face Crea...,16.66,4.3,"Aqua/Water, Cyclohexasiloxane, Glycerin, Paraf...",1.0,1.0,1.0,1.0,1.0,-0.031734,-2.106823
1,118,118,Moisturizer,L'Oreal Paris,L'Oreal Paris Triple Active Day Moisturiser Dr...,4.33,4.4,"Aqua/Water, Glycerin, Pentaerythrityl Tetraeth...",0.0,1.0,0.0,0.0,1.0,-4.311086,-3.011751
2,119,119,Moisturizer,L'Oreal Paris,L'Oreal Paris Age Perfect Rehydrating Day Crea...,7.99,4.5,"Aqua / Water , Caprylic/Capric Triglyceride , ...",0.0,1.0,0.0,0.0,0.0,-4.253574,-1.708941
3,123,123,Moisturizer,L'Oreal Paris,L'Oreal Paris Revitalift Face Contours and Nec...,9.99,4.0,"Aqua / Water, Dimethicone, Simmondsia Chinensi...",1.0,1.0,1.0,1.0,1.0,2.518959,0.527802
4,125,125,Moisturizer,L'Oreal Paris,L'Oreal Paris Wrinkle Decrease Night Cream 50ml,9.00,4.7,"Aqua / Water , Cyclohexasiloxane , Glycerin , ...",1.0,1.0,1.0,1.0,1.0,-1.447590,-2.117161


# Bokeh

In [1]:
from bokeh.io import show, curdoc, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Select, Paragraph, TextInput
from bokeh.layouts import widgetbox, column, row
from ipywidgets import interact

In [2]:
import pandas as pd

df_bokeh = pd.read_csv("Data/cosmetic_all_comb.csv")

In [3]:
df_bokeh.head()

,level_0,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
0,0,0,Cleanser_Combination,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0,25.145235,-126.716965
1,1,1,Cleanser_Combination,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0,106.132440,35.102200
2,2,2,Cleanser_Combination,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0,148.667880,123.158810
3,3,3,Cleanser_Combination,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0,150.261380,-13.898534
4,4,4,Cleanser_Combination,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0,53.702545,5.013023


In [9]:
df_bokeh.drop(columns=["level_0", "index"], axis=1, inplace=True)

In [10]:
df_bokeh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5349 entries, 0 to 5348
Data columns (total 13 columns):
Label          5349 non-null object
brand          5349 non-null object
name           5349 non-null object
price          5349 non-null object
rank           5349 non-null float64
ingredients    5349 non-null object
Combination    5349 non-null float64
Dry            5349 non-null float64
Normal         5349 non-null float64
Oily           5349 non-null float64
Sensitive      5349 non-null float64
X              5349 non-null float64
Y              5349 non-null float64
dtypes: float64(8), object(5)
memory usage: 543.4+ KB


In [15]:
product_category = ['Cleanser',
 'Treatment',
 'Serum',
 'Oils',
 'Toners',
 'Moisturizer',
 'Night Treatments',
 'Face Mask',
 'Eye cream',
 'Sun protect']
skin_type = ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']

In [16]:
output_notebook()

Loading BokehJS ...

In [17]:
# make a source and scatter bokeh plot  
source = ColumnDataSource(df_bokeh)
plot = figure(x_axis_label = 'T-SNE 1', y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)
plot.circle(x = 'X', y = 'Y', source = source, 
            size = 10, color = '#FF7373', alpha = .8)

plot.background_fill_color = "beige"
plot.background_fill_alpha = 0.2

# add hover tool
hover = HoverTool(tooltips = [
        ('Item', '@name'),
        ('brand', '@brand'),
        ('Price', '$ @price'),
        ('Rank', '@rank')])
plot.add_tools(hover)

In [18]:
def update(select1 = product_category[0], select2 = skin_type[0]):
    a_b = select1 + '_' + select2
    new_data = {
        'X' : df_bokeh[df_bokeh['Label'] == a_b]['X'],
        'Y' : df_bokeh[df_bokeh['Label'] == a_b]['Y'],
        'name' : df_bokeh[df_bokeh['Label'] == a_b]['name'],
        'brand' : df_bokeh[df_bokeh['Label'] == a_b]['brand'],
        'price' : df_bokeh[df_bokeh['Label'] == a_b]['price'],
        'rank' : df_bokeh[df_bokeh['Label'] == a_b]['rank'],
    }
    source.data = new_data
    push_notebook()

In [19]:
# interact the plot with callback 

interact(update, select1 = product_category, select2 = skin_type)
show(plot, notebook_handle = True)

interactive(children=(Dropdown(description='select1', options=('Cleanser', 'Treatment', 'Serum', 'Oils', 'Tone…

# PCA

In [64]:
df_pca = pd.read_csv("Data/cosmetic_all.csv")
df_pca.head()

,Unnamed: 0,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,Cleanser,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0
1,1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0
2,2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0
3,3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0
4,4,Cleanser,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0


In [65]:
df_pca.drop("Unnamed: 0", axis=True, inplace=True)

In [66]:
# Tokenizing ingredients
ingredient_idx_pca = {}
corpus_pca = []
idx_pca = 0

for i in range(len(df_pca)):
    ingredients_pca = df_pca["ingredients"][i]
    ingredients_lower_pca = ingredients_pca.lower() # lower the ingredients
    tokens_pca = ingredients_lower_pca.split(",") # split the text
    corpus_pca.append(tokens_pca) # text added to corpus
    for ingredient in tokens_pca:
        if ingredient not in ingredient_idx_pca: # if no duplicates 
            ingredient_idx_pca[ingredient] = idx_pca # put it in the dictionary with index
            idx_pca += 1 # value of index will increase one by one

In [70]:
# making document-term matrix
# each cosmetic product will correspond to one document
# first, make an empty matrix 
A = len(df_pca)
B = len(ingredient_idx_pca)

# fill it with zeros
matrix_pca = np.zeros((A,B))

In [71]:
def count_tokens(tokens_pca):
    x = np.zeros(B)
    for ingredient in tokens_pca:
        idx_pca = ingredient_idx_pca[ingredient]
        x[idx_pca] = 1
    return x

In [72]:
i = 0
for tokens in corpus_pca:
    matrix_pca[i,:] = count_tokens(tokens_pca)
    i += 1

In [73]:
matrix_pca.shape

(1699, 7822)

In [81]:
pca_df = pd.DataFrame(matrix_pca)

In [87]:
# pca_df = pca_df + 1
pca_flat = matrix_pca.reshape((1, -1))

In [88]:
from sklearn.decomposition import PCA

fit = PCA()
pca = fit.fit_transform(pca_flat)
pca

/Users/lynguyenova/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/pca.py:445: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)


array([[0.]])

# Top ingredients for Dry Skin

In [21]:
# dry skin dataframe

dry_df = df.loc[df["Dry"] == 1]
dry_df = dry_df.reset_index()

In [22]:
dry_df.head()

,level_0,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,0,Cleanser,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0
1,1,1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0
2,2,2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0
3,3,3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0
4,4,4,Cleanser,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0


In [25]:
# Tokenizing ingredients
ingredient_idx_dry = {}
corpus_dry = []
idx_dry = 0

In [26]:
for i in range(len(dry_df)):
    ingredients_dry = dry_df["ingredients"][i]
    ingredients_lower_dry = ingredients_dry.lower() # lower the ingredients
    tokens_dry = ingredients_lower_dry.split(",") # split the text
    corpus_dry.append(tokens_dry) # text added to corpus
    for ingredient in tokens_dry:
        if ingredient not in ingredient_idx_dry: # if no duplicates 
            ingredient_idx_dry[ingredient] = idx_dry # put it in the dictionary with index
            idx_dry += 1 # value of index will increase one by one

In [27]:
import numpy as np
# making document-term matrix
# each cosmetic product will correspond to one document
# first, make an empty matrix 
E = len(dry_df)
F = len(ingredient_idx_dry)

# fill it with zeros
matrix_dry = np.zeros((E,F))

In [28]:
matrix_dry.shape

(1091, 6196)

In [29]:
# 1 if the ingredient was in the product
# 0 if the ingredient was not in the product
def count_tokens(tokens):
    x = np.zeros(F)
    for ingredient in tokens_dry:
        idx = ingredient_idx_dry[ingredient]
        x[idx] = 1
    return x

In [30]:
# apply to the corpus
# start making the cosmetic-term matrix
i = 0
for tokens in corpus_dry:
    matrix_dry[i,:] = count_tokens(tokens)
    i += 1

In [56]:
# import itertools

# d = dict(itertools.islice(ingredient_idx_dry.items(), 150))

In [31]:
# import operator

sorted_d = sorted(ingredient_idx_dry.items(), key=operator.itemgetter(1), reverse=True)

In [32]:
top_d = sorted_d[:20]

In [33]:
top_d

[('visit the dermaflash boutique', 6195),
 (' ozonized oryza sativa (rice) callus culture extract', 6194),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 6193),
 (' quaternium-18 bentonite', 6192),
 (' and avobenzone 3% deliver broad-spectrum sun protection.', 6191),
 (' octisalate 5%', 6190),
 ('-homosalate 10%', 6189),
 (' c30-45 alkyl cetearyl dimethicone crosspolymer', 6188),
 (' ornithine hcl', 6187),
 (' caprylic/caprlc triglyceride', 6186),
 (' trisodium ethylenediamine disuccinate.', 6185),
 (' tetrahexlydecyl ascorbate', 6184),
 (' acrylates/octylacrylamide copolymer', 6183),
 (' diisopropyl adipate', 6182),
 ('sd alcohol 40', 6181),
 (' ubiquinone (coenzyme q-10).', 6180),
 (' purified water', 6179),
 (' peg-8 methyl triethoxysilane', 6178),
 (' cetyl peg/ppg 10/1 dimethicone', 6177),
 (' camellia sinensis (green tea) polyphenols', 6176)]

In [34]:
top_d = pd.DataFrame(top_d)

In [35]:
top_d

,0,1
0,visit the dermaflash boutique,6195
1,ozonized oryza sativa (rice) callus culture e...,6194
2,chlorhexidine digluconate. [+/-: iron oxides ...,6193
3,quaternium-18 bentonite,6192
4,and avobenzone 3% deliver broad-spectrum sun ...,6191
5,octisalate 5%,6190
6,-homosalate 10%,6189
7,c30-45 alkyl cetearyl dimethicone crosspolymer,6188
8,ornithine hcl,6187
9,caprylic/caprlc triglyceride,6186


# Top ingredients for Combination Skin

In [36]:
# dry skin dataframe

combi_df = df.loc[df["Combination"] == 1]
combi_df = combi_df.reset_index()

In [38]:
# Tokenizing ingredients
ingredient_idx_combi = {}
corpus_combi = []
idx_combi = 0

In [39]:
for i in range(len(combi_df)):
    ingredients_combi = combi_df["ingredients"][i]
    ingredients_lower_combi = ingredients_combi.lower() # lower the ingredients
    tokens_combi = ingredients_lower_combi.split(",") # split the text
    corpus_combi.append(tokens_combi) # text added to corpus
    for ingredient in tokens_combi:
        if ingredient not in ingredient_idx_combi: # if no duplicates 
            ingredient_idx_combi[ingredient] = idx_combi # put it in the dictionary with index
            idx_combi += 1 # value of index will increase one by one

In [40]:
C = len(combi_df)
D = len(ingredient_idx_combi)

# fill it with zeros
matrix_combi = np.zeros((C,D))

In [41]:
def count_tokens(tokens):
    x = np.zeros(D)
    for ingredient in tokens_combi:
        idx = ingredient_idx_combi[ingredient]
        x[idx] = 1
    return x

In [42]:
i = 0
for tokens in corpus_combi:
    matrix_combi[i,:] = count_tokens(tokens)
    i += 1

In [43]:
sorted_c = sorted(ingredient_idx_combi.items(), key=operator.itemgetter(1), reverse=True)

In [44]:
top_c = sorted_c[:20]

In [45]:
top_c

[('visit the dermaflash boutique', 6368),
 (' ozonized oryza sativa (rice) callus culture extract', 6367),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 6366),
 (' quaternium-18 bentonite', 6365),
 (' and avobenzone 3% deliver broad-spectrum sun protection.', 6364),
 (' octisalate 5%', 6363),
 ('-homosalate 10%', 6362),
 (' c30-45 alkyl cetearyl dimethicone crosspolymer', 6361),
 (' ornithine hcl', 6360),
 (' caprylic/caprlc triglyceride', 6359),
 (' trisodium ethylenediamine disuccinate.', 6358),
 (' tetrahexlydecyl ascorbate', 6357),
 (' acrylates/octylacrylamide copolymer', 6356),
 ('sd alcohol 40', 6355),
 (' ubiquinone (coenzyme q-10).', 6354),
 (' purified water', 6353),
 (' peg-8 methyl triethoxysilane', 6352),
 (' cetyl peg/ppg 10/1 dimethicone', 6351),
 (' camellia sinensis (green tea) polyphenols', 6350),
 ('ascorbic acid', 6349)]

In [46]:
top_c = pd.DataFrame(top_c)

In [47]:
top_c

,0,1
0,visit the dermaflash boutique,6368
1,ozonized oryza sativa (rice) callus culture e...,6367
2,chlorhexidine digluconate. [+/-: iron oxides ...,6366
3,quaternium-18 bentonite,6365
4,and avobenzone 3% deliver broad-spectrum sun ...,6364
5,octisalate 5%,6363
6,-homosalate 10%,6362
7,c30-45 alkyl cetearyl dimethicone crosspolymer,6361
8,ornithine hcl,6360
9,caprylic/caprlc triglyceride,6359


# Top ingredients for Normal Skin

In [48]:
norm_df = df.loc[df["Normal"] == 1]
norm_df = norm_df.reset_index()
# norm_df.drop("Unnamed: 0", axis=1, inplace=True

In [49]:
norm_df.head()

,level_0,index,Label,brand,name,price,rank,ingredients,Combination,Dry,Normal,Oily,Sensitive
0,0,0,Cleanser,Fresh,Soy Face Cleanser,30.0,4.2141,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle...",1.0,1.0,1.0,1.0,1.0
1,1,1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,85.0,4.3369,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ...",1.0,1.0,1.0,1.0,1.0
2,2,2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,29.0,4.4780,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce...",1.0,1.0,1.0,1.0,1.0
3,3,3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,47.0,4.5974,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda...",1.0,1.0,1.0,1.0,1.0
4,4,4,Cleanser,Oskia,Renaissance Cleansing Gel,35.0,4.3985,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ...",1.0,1.0,1.0,0.0,0.0


In [50]:
# Tokenizing ingredients
ingredient_idx_norm = {}
corpus_norm = []
idx_norm = 0

for i in range(len(norm_df)):
    ingredients_norm = norm_df["ingredients"][i]
    ingredients_lower_norm = ingredients_norm.lower() # lower the ingredients
    tokens_norm = ingredients_lower_norm.split(",") # split the text
    corpus_norm.append(tokens_norm) # text added to corpus
    for ingredient in tokens_norm:
        if ingredient not in ingredient_idx_norm: # if no duplicates 
            ingredient_idx_norm[ingredient] = idx_norm # put it in the dictionary with index
            idx_norm += 1 # value of index will increase one by one
            
H = len(norm_df)
L = len(ingredient_idx_norm)

# fill it with zeros
matrix_norm = np.zeros((H,L))

In [51]:
def count_tokens(tokens):
    x = np.zeros(L)
    for ingredient in tokens_norm:
        idx = ingredient_idx_norm[ingredient]
        x[idx] = 1
    return x

In [52]:
i = 0
for tokens in corpus_norm:
    matrix_norm[i,:] = count_tokens(tokens)
    i += 1

In [53]:
sorted_n = sorted(ingredient_idx_norm.items(), key=operator.itemgetter(1), reverse=True)

In [54]:
top_n = sorted_n[:20]

In [55]:
top_n

[('visit the dermaflash boutique', 6322),
 (' ozonized oryza sativa (rice) callus culture extract', 6321),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 6320),
 (' quaternium-18 bentonite', 6319),
 (' and avobenzone 3% deliver broad-spectrum sun protection.', 6318),
 (' octisalate 5%', 6317),
 ('-homosalate 10%', 6316),
 (' c30-45 alkyl cetearyl dimethicone crosspolymer', 6315),
 (' ornithine hcl', 6314),
 (' caprylic/caprlc triglyceride', 6313),
 (' trisodium ethylenediamine disuccinate.', 6312),
 (' tetrahexlydecyl ascorbate', 6311),
 (' acrylates/octylacrylamide copolymer', 6310),
 ('sd alcohol 40', 6309),
 (' ubiquinone (coenzyme q-10).', 6308),
 (' purified water', 6307),
 (' peg-8 methyl triethoxysilane', 6306),
 (' cetyl peg/ppg 10/1 dimethicone', 6305),
 (' camellia sinensis (green tea) polyphenols', 6304),
 ('ascorbic acid', 6303)]

In [56]:
top_n = pd.DataFrame(top_n)

In [57]:
top_n

,0,1
0,visit the dermaflash boutique,6322
1,ozonized oryza sativa (rice) callus culture e...,6321
2,chlorhexidine digluconate. [+/-: iron oxides ...,6320
3,quaternium-18 bentonite,6319
4,and avobenzone 3% deliver broad-spectrum sun ...,6318
5,octisalate 5%,6317
6,-homosalate 10%,6316
7,c30-45 alkyl cetearyl dimethicone crosspolymer,6315
8,ornithine hcl,6314
9,caprylic/caprlc triglyceride,6313


# Top ingredients for Oily Skin

In [58]:
oil_df = df.loc[df["Oily"] == 1]
oil_df = oil_df.reset_index()
# norm_df.drop("Unnamed: 0", axis=1, inplace=True

In [59]:
# Tokenizing ingredients
ingredient_idx_oil = {}
corpus_oil = []
idx_oil = 0

for i in range(len(oil_df)):
    ingredients_oil = oil_df["ingredients"][i]
    ingredients_lower_oil = ingredients_oil.lower() # lower the ingredients
    tokens_oil = ingredients_lower_oil.split(",") # split the text
    corpus_oil.append(tokens_oil) # text added to corpus
    for ingredient in tokens_oil:
        if ingredient not in ingredient_idx_oil: # if no duplicates 
            ingredient_idx_oil[ingredient] = idx_oil # put it in the dictionary with index
            idx_oil += 1 # value of index will increase one by one
            
O = len(oil_df)
P = len(ingredient_idx_oil)

# fill it with zeros
matrix_oil = np.zeros((O,P))

In [60]:
def count_tokens(tokens):
    x = np.zeros(P)
    for ingredient in tokens_oil:
        idx = ingredient_idx_oil[ingredient]
        x[idx] = 1
    return x

In [61]:
i = 0
for tokens in corpus_oil:
    matrix_oil[i,:] = count_tokens(tokens)
    i += 1

In [62]:
sorted_o = sorted(ingredient_idx_oil.items(), key=operator.itemgetter(1), reverse=True)

In [63]:
top_o = sorted_o[:20]

In [64]:
top_o

[('visit the dermaflash boutique', 6082),
 (' ozonized oryza sativa (rice) callus culture extract', 6081),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 6080),
 (' quaternium-18 bentonite', 6079),
 (' and avobenzone 3% deliver broad-spectrum sun protection.', 6078),
 (' octisalate 5%', 6077),
 ('-homosalate 10%', 6076),
 (' c30-45 alkyl cetearyl dimethicone crosspolymer', 6075),
 (' ornithine hcl', 6074),
 (' passiflora incarnata seed oil', 6073),
 (' caprylic/caprlc triglyceride', 6072),
 (' trisodium ethylenediamine disuccinate.', 6071),
 (' tetrahexlydecyl ascorbate', 6070),
 (' acrylates/octylacrylamide copolymer', 6069),
 ('sd alcohol 40', 6068),
 (' ubiquinone (coenzyme q-10).', 6067),
 (' purified water', 6066),
 (' peg-8 methyl triethoxysilane', 6065),
 (' cetyl peg/ppg 10/1 dimethicone', 6064),
 (' camellia sinensis (green tea) polyphenols', 6063)]

In [65]:
top_o = pd.DataFrame(top_o)

In [66]:
top_o

,0,1
0,visit the dermaflash boutique,6082
1,ozonized oryza sativa (rice) callus culture e...,6081
2,chlorhexidine digluconate. [+/-: iron oxides ...,6080
3,quaternium-18 bentonite,6079
4,and avobenzone 3% deliver broad-spectrum sun ...,6078
5,octisalate 5%,6077
6,-homosalate 10%,6076
7,c30-45 alkyl cetearyl dimethicone crosspolymer,6075
8,ornithine hcl,6074
9,passiflora incarnata seed oil,6073


# Top ingredients for Sensitive Skin

In [67]:
sen_df = df.loc[df["Normal"] == 1]
sen_df = sen_df.reset_index()
# sen_df.drop("Unnamed: 0", axis=1, inplace=True

In [68]:
# Tokenizing ingredients
ingredient_idx_sen = {}
corpus_sen = []
idx_sen = 0

for i in range(len(sen_df)):
    ingredients_sen = sen_df["ingredients"][i]
    ingredients_lower_sen = ingredients_sen.lower() # lower the ingredients
    tokens_sen = ingredients_lower_sen.split(",") # split the text
    corpus_sen.append(tokens_sen) # text added to corpus
    for ingredient in tokens_sen:
        if ingredient not in ingredient_idx_sen: # if no duplicates 
            ingredient_idx_sen[ingredient] = idx_sen # put it in the dictionary with index
            idx_sen += 1 # value of index will increase one by one
            
S = len(sen_df)
T = len(ingredient_idx_sen)

# fill it with zeros
matrix_sen = np.zeros((S,T))

In [69]:
def count_tokens(tokens):
    x = np.zeros(T)
    for ingredient in tokens_sen:
        idx = ingredient_idx_sen[ingredient]
        x[idx] = 1
    return x

In [70]:
i = 0
for tokens in corpus_sen:
    matrix_sen[i,:] = count_tokens(tokens)
    i += 1

In [71]:
sorted_s = sorted(ingredient_idx_sen.items(), key=operator.itemgetter(1), reverse=True)

In [72]:
top_s = sorted_s[:20]

In [73]:
top_s

[('visit the dermaflash boutique', 6322),
 (' ozonized oryza sativa (rice) callus culture extract', 6321),
 (' chlorhexidine digluconate. [+/-: iron oxides (ci 77491', 6320),
 (' quaternium-18 bentonite', 6319),
 (' and avobenzone 3% deliver broad-spectrum sun protection.', 6318),
 (' octisalate 5%', 6317),
 ('-homosalate 10%', 6316),
 (' c30-45 alkyl cetearyl dimethicone crosspolymer', 6315),
 (' ornithine hcl', 6314),
 (' caprylic/caprlc triglyceride', 6313),
 (' trisodium ethylenediamine disuccinate.', 6312),
 (' tetrahexlydecyl ascorbate', 6311),
 (' acrylates/octylacrylamide copolymer', 6310),
 ('sd alcohol 40', 6309),
 (' ubiquinone (coenzyme q-10).', 6308),
 (' purified water', 6307),
 (' peg-8 methyl triethoxysilane', 6306),
 (' cetyl peg/ppg 10/1 dimethicone', 6305),
 (' camellia sinensis (green tea) polyphenols', 6304),
 ('ascorbic acid', 6303)]

In [74]:
top_s = pd.DataFrame(top_s)

In [75]:
top_s

,0,1
0,visit the dermaflash boutique,6322
1,ozonized oryza sativa (rice) callus culture e...,6321
2,chlorhexidine digluconate. [+/-: iron oxides ...,6320
3,quaternium-18 bentonite,6319
4,and avobenzone 3% deliver broad-spectrum sun ...,6318
5,octisalate 5%,6317
6,-homosalate 10%,6316
7,c30-45 alkyl cetearyl dimethicone crosspolymer,6315
8,ornithine hcl,6314
9,caprylic/caprlc triglyceride,6313


# Content-based recommender system

In [3]:
pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7821 sha256=d019492b31ed1bd535719f329f49a0f052024f5fbd7f490dac50ee6aa0639e65
  Stored in directory: /Users/lynguyenova/Library/Caches/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
df_doc = df[["Label", "brand","name", "ingredients"]]

In [15]:
df_doc

,Label,brand,name,ingredients
0,Cleanser,Fresh,Soy Face Cleanser,"Aqua (Water), Coco-Glucoside, Glycerin, Butyle..."
1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,"Paraffinum Liquidum, Peg-30 Lanolin, Cetearyl ..."
2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,"Ethylhexyl Palmitate, Caprylic/Capric Triglyce..."
3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,"Vitis Vinifera (Grape) Seed Oil, Prunus Amygda..."
4,Cleanser,Oskia,Renaissance Cleansing Gel,"Caprylic/Capric Triglyceride, Glycerin, Aqua, ..."
...,...,...,...,...
1694,Sun protect,KORRES,Yoghurt Nourishing Fluid Veil Face Sunscreen B...,"Water, Alcohol Denat., Potassium Cetyl Phospha..."
1695,Sun protect,KATE SOMERVILLE,Daily Deflector™ Waterlight Broad Spectrum SPF...,"Water, Isododecane, Dimethicone, Butyloctyl Sa..."
1696,Sun protect,VITA LIBERATA,Self Tan Dry Oil SPF 50,"Water, Dihydroxyacetone, Glycerin, Sclerocarya..."
1697,Sun protect,ST. TROPEZ TANNING ESSENTIALS,Pro Light Self Tan Bronzing Mist,"Water, Dihydroxyacetone, Propylene Glycol, PPG..."


In [16]:
# initializing the new column
df_doc['Key_words'] = ""

for index, row in df_doc.iterrows():
    ingredients = row['ingredients']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(ingredients)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding products
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the ingredients column
df_doc.drop(columns = ['ingredients'], inplace = True)

/Users/lynguyenova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df_doc.head()

,Label,brand,name,Key_words
0,Cleanser,Fresh,Soy Face Cleanser,"[flower, extract, caramel, rosa, damascena, wa..."
1,Cleanser,Eve Lom,Cleanser 200ml - Includes 2 Muslin Cloths,"[30, lanolin, eucalyptus, globulus, leaf, oil,..."
2,Cleanser,Drunk Elephant,Slaai Makeup Melting Butter Cleanser,"[2, caprate, actinidia, chinensis, glycerin, h..."
3,Cleanser,Emma Hardie,Amazing Face Moringa Cleansing Balm,"[citronellol, geraniol, bitter, orange, citral..."
4,Cleanser,Oskia,Renaissance Cleansing Gel,"[msm, ),, citronellol, **,, dehydroacetic, aci..."


In [18]:
# lower brands' names
df_doc['brand'] = df_doc['brand'].map(lambda x: x.lower())

df_doc['Label'] = df_doc['Label'].map(lambda x: x.lower())


/Users/lynguyenova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lynguyenova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:
df_doc.head()

,Label,brand,name,Key_words
0,cleanser,fresh,Soy Face Cleanser,"[flower, extract, caramel, rosa, damascena, wa..."
1,cleanser,eve lom,Cleanser 200ml - Includes 2 Muslin Cloths,"[30, lanolin, eucalyptus, globulus, leaf, oil,..."
2,cleanser,drunk elephant,Slaai Makeup Melting Butter Cleanser,"[2, caprate, actinidia, chinensis, glycerin, h..."
3,cleanser,emma hardie,Amazing Face Moringa Cleansing Balm,"[citronellol, geraniol, bitter, orange, citral..."
4,cleanser,oskia,Renaissance Cleansing Gel,"[msm, ),, citronellol, **,, dehydroacetic, aci..."


In [20]:

df_doc.set_index('name', inplace = True)
df_doc.head()

,Label,brand,Key_words
name,,,
Soy Face Cleanser,cleanser,fresh,"[flower, extract, caramel, rosa, damascena, wa..."
Cleanser 200ml - Includes 2 Muslin Cloths,cleanser,eve lom,"[30, lanolin, eucalyptus, globulus, leaf, oil,..."
Slaai Makeup Melting Butter Cleanser,cleanser,drunk elephant,"[2, caprate, actinidia, chinensis, glycerin, h..."
Amazing Face Moringa Cleansing Balm,cleanser,emma hardie,"[citronellol, geraniol, bitter, orange, citral..."
Renaissance Cleansing Gel,cleanser,oskia,"[msm, ),, citronellol, **,, dehydroacetic, aci..."


In [21]:
df_doc['bag_of_words'] = ''
columns = df_doc.columns
for index, row in df_doc.iterrows():
    words = ''
    for col in columns:
        if col != 'brand':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df_doc.drop(columns = [col for col in df_doc.columns if col!= 'bag_of_words'], inplace = True)

/Users/lynguyenova/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_doc.head()

,bag_of_words
name,
Soy Face Cleanser,c l e a n s e r fresh flower extract caramel r...
Cleanser 200ml - Includes 2 Muslin Cloths,c l e a n s e r eve lom 30 lanolin eucalyptus ...
Slaai Makeup Melting Butter Cleanser,c l e a n s e r drunk elephant 2 caprate actin...
Amazing Face Moringa Cleansing Balm,c l e a n s e r emma hardie citronellol gerani...
Renaissance Cleansing Gel,"c l e a n s e r oskia msm ), citronellol **, d..."


In [25]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df_doc['bag_of_words'])

# generating the cosine  similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [26]:
cosine_sim

array([[1.        , 0.14233614, 0.23144695, ..., 0.25125945, 0.16903085,
        0.        ],
       [0.14233614, 1.        , 0.04169382, ..., 0.07242068, 0.0541332 ,
        0.        ],
       [0.23144695, 0.04169382, 1.        , ..., 0.1177603 , 0.04401188,
        0.        ],
       ...,
       [0.25125945, 0.07242068, 0.1177603 , ..., 1.        , 0.15289416,
        0.        ],
       [0.16903085, 0.0541332 , 0.04401188, ..., 0.15289416, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [29]:
indices = pd.Series(df_doc.index)
indices[:5]

0                            Soy Face Cleanser
1    Cleanser 200ml - Includes 2 Muslin Cloths
2         Slaai Makeup Melting Butter Cleanser
3          Amazing Face Moringa Cleansing Balm
4                    Renaissance Cleansing Gel
Name: name, dtype: object

In [27]:
# function that takes in product title as input and returns the top 10 recommended products
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_products = []
    
    # gettin the index of the products that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar products
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching products
    for i in top_10_indexes:
        recommended_products.append(list(df_doc.index)[i])
        
    return recommended_products

In [30]:
recommendations('Renaissance Cleansing Gel')

['Renewing Rose Cleanser',
 'Cashmere Cleanser',
 'Moor Cream Cleanser',
 'Perfect Canvas Jelly Oil Cleanser',
 'Multi-Miracle Glow Cleansing Balm',
 'Amazing Face Moringa Cleansing Balm',
 'Lancôme Absolue Brightening Oil in Gel Cleanser 125ml',
 'Multi Miracle Glow Cleansing Balm',
 'Nourishing Cleansing Balm',
 'Deep Cleansing Mask']

In [31]:
M = recommendations('Renaissance Cleansing Gel')

In [32]:
M = pd.DataFrame(M)

In [33]:
M

,0
0,Renewing Rose Cleanser
1,Cashmere Cleanser
2,Moor Cream Cleanser
3,Perfect Canvas Jelly Oil Cleanser
4,Multi-Miracle Glow Cleansing Balm
5,Amazing Face Moringa Cleansing Balm
6,Lancôme Absolue Brightening Oil in Gel Cleanse...
7,Multi Miracle Glow Cleansing Balm
8,Nourishing Cleansing Balm
9,Deep Cleansing Mask
